Para empezar la limpieza de datos empezaros primero cargando librerias e ir limpiando columna a columna las cosas que no nos cuadren.

In [54]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Queremos limpiar nulls y datos que no concuerdan

In [55]:
df = pd.read_csv("../data/raw/bgg_dataset.csv",sep=";")

In [56]:
df["Complexity Average"] = df["Complexity Average"].str.replace(",",".").astype(float)
df["Rating Average"] = df["Rating Average"].str.replace(",",".").astype(float)
media_valores_año = int(np.mean(df.loc[df["Year Published"] >= 0, "Year Published"]))
df.loc[df["Year Published"] < 1, "Year Published"] = media_valores_año
df.loc[df["Year Published"].isna(), "Year Published"] = media_valores_año
df["Year Published"] = df["Year Published"].astype(int)


Quitamos de la fila de Domains los null como no definidos.

In [57]:
df.loc[df["Domains"].isnull(), "Domains"] = "Not Defined"

In [58]:
df["Domains"].unique()

array(['Strategy Games, Thematic Games', 'Strategy Games',
       'Thematic Games', 'Strategy Games, Wargames',
       'Thematic Games, Wargames', 'Family Games, Strategy Games',
       'Customizable Games, Thematic Games',
       'Abstract Games, Family Games', 'Customizable Games',
       'Family Games', 'Party Games', 'Customizable Games, Wargames',
       'Wargames', 'Party Games, Thematic Games', 'Abstract Games',
       'Customizable Games, Strategy Games',
       'Family Games, Thematic Games', 'Family Games, Party Games',
       'Abstract Games, Strategy Games', "Children's Games, Family Games",
       'Not Defined', 'Party Games, Strategy Games', "Children's Games",
       "Children's Games, Party Games",
       'Abstract Games, Customizable Games',
       'Family Games, Strategy Games, Thematic Games',
       'Family Games, Party Games, Thematic Games',
       'Strategy Games, Thematic Games, Wargames',
       'Abstract Games, Party Games', "Abstract Games, Children's Games",

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20343 entries, 0 to 20342
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  20327 non-null  float64
 1   Name                20343 non-null  object 
 2   Year Published      20343 non-null  int32  
 3   Min Players         20343 non-null  int64  
 4   Max Players         20343 non-null  int64  
 5   Play Time           20343 non-null  int64  
 6   Min Age             20343 non-null  int64  
 7   Users Rated         20343 non-null  int64  
 8   Rating Average      20343 non-null  float64
 9   BGG Rank            20343 non-null  int64  
 10  Complexity Average  20343 non-null  float64
 11  Owned Users         20320 non-null  float64
 12  Mechanics           18745 non-null  object 
 13  Domains             20343 non-null  object 
dtypes: float64(4), int32(1), int64(6), object(3)
memory usage: 2.1+ MB


In [67]:
df['ID'].dropna(inplace=True)

In [65]:
df["ID"].isnull().value_counts()

False    20327
True        16
Name: ID, dtype: int64

In [32]:
df.loc[df['ID'].min()]

ID                                                             161936.0
Name                                          Pandemic Legacy: Season 1
Year Published                                                     2015
Min Players                                                           2
Max Players                                                           4
Play Time                                                            60
Min Age                                                              13
Users Rated                                                       41643
Rating Average                                                     8.61
BGG Rank                                                              2
Complexity Average                                                 2.84
Owned Users                                                     65294.0
Mechanics             Action Points, Cooperative Game, Hand Manageme...
Domains                                  Strategy Games, Themati

In [69]:
df.loc[df["ID"].isnull()]

,ID,Name,Year Published,Min Players,Max Players,Play Time,Min Age,Users Rated,Rating Average,BGG Rank,Complexity Average,Owned Users,Mechanics,Domains
10776,NaN,Ace of Aces: Jet Eagles,1990,2,2,20,10,110,6.26,10778,2.00,NaN,NaN,Not Defined
10835,NaN,Die Erben von Hoax,1999,3,8,45,12,137,6.05,10837,2.00,NaN,NaN,Not Defined
11152,NaN,Rommel in North Africa: The War in the Desert ...,1986,2,2,0,12,53,6.76,11154,4.00,NaN,NaN,Not Defined
11669,NaN,Migration: A Story of Generations,2012,2,4,30,12,49,7.20,11671,2.00,NaN,NaN,Not Defined
12649,NaN,Die Insel der steinernen Wachter,2009,2,4,120,12,49,6.73,12651,3.00,NaN,NaN,Not Defined
12764,NaN,Dragon Ball Z TCG (2014 edition),2014,2,2,20,8,33,7.03,12766,2.50,NaN,NaN,Not Defined
13282,NaN,Dwarfest,2014,2,6,45,12,82,6.13,13284,1.75,NaN,NaN,Not Defined
13984,NaN,Hus,1986,2,2,40,0,38,6.28,13986,2.00,NaN,NaN,Not Defined
14053,NaN,Contrario 2,2006,2,12,0,14,37,6.30,14055,1.00,NaN,NaN,Not Defined
14663,NaN,Warage: Extended Edition,2017,2,6,90,10,49,7.64,14665,3.00,NaN,NaN,Not Defined
